In [2]:
import requests
import json
import pandas as pd

### Skl matrix, 1 if station l is closer than 540s to neighbor k

In [3]:
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones_barrios.json")
distances_stations_ntas_db = json.loads(response.text)
neighbors = distances_stations_ntas_db["42 South Street"].keys() 
df = pd.DataFrame(distances_stations_ntas_db).transpose()

In [4]:
skl = df

In [5]:
for i in range(len(df)):
    for n in neighbors:
        if df[n][i]<=540.0:skl[n][i]=1
        else:skl[n][i]=0

In [12]:
skl.to_csv("../aux_data/skl.csv")

### Distances between station l and neighbor k

In [13]:
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones_barrios.json")
distances_stations_ntas_db = json.loads(response.text) 
dlk = pd.DataFrame(distances_stations_ntas_db).transpose()
dlk.to_csv("/home/vant/Documentos/mySpace/cuarto/opt/projectI/Optimization_NY_Fire_Departments/aux_data/dlk.csv")

### Cost of hazmat in station l in shift i

In [35]:
d = pd.read_csv("/home/vant/Documentos/mySpace/cuarto/opt/projectI/Optimization_NY_Fire_Departments/aux_data/dlk.csv")
b = pd.read_csv("/home/vant/Documentos/mySpace/cuarto/opt/projectI/Optimization_NY_Fire_Departments/aux_data/bijk.csv").drop(['Unnamed: 0'],axis=1)

In [36]:
aux = list(d['Unnamed: 0'])
d=d.drop(['Unnamed: 0'],axis=1)
d = d.set_axis(aux)

In [49]:
hazmat_1 = {b['nta'][i]:b['ha'][i] for i in range(len(b)) if b['shift'][i]==1}
hazmat_0 = {b['nta'][i]:b['ha'][i] for i in range(len(b)) if b['shift'][i]==0}

In [65]:
costs_ha1 = []
for k,v in hazmat_1.items():
    acum = 0
    for i in range(len(d)):
        value = d.iloc[i][k]*1/v
        acum+=value
    costs_ha1.append(acum/45)

costs_ha2 = []
for k,v in hazmat_0.items():
    acum = 0
    for i in range(len(d)):
        value = d.iloc[i][k]*1/v
        acum+=value
    costs_ha2.append(acum/44)

costs_ha=costs_ha1+costs_ha2   

### Cost of squads in station l in shift i

In [68]:
squads_1 = {b['nta'][i]:b['squads'][i] for i in range(len(b)) if b['shift'][i]==1}
squads_0 = {b['nta'][i]:b['squads'][i] for i in range(len(b)) if b['shift'][i]==0}

In [69]:
costs_squads1 = []
for k,v in squads_1.items():
    acum = 0
    for i in range(len(d)):
        value = d.iloc[i][k]*1/v
        acum+=value
    costs_squads1.append(acum/45)

costs_squads2 = []
for k,v in squads_0.items():
    acum = 0
    for i in range(len(d)):
        value = d.iloc[i][k]*1/v
        acum+=value
    costs_squads2.append(acum/44)

costs_squads=costs_squads1+costs_squads2 

/tmp/ipykernel_49560/2225570417.py:13: RuntimeWarning: divide by zero encountered in scalar divide
  value = d.iloc[i][k]*1/v


### Neighbor k belongs to borough v

In [3]:
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/barrios.json")
ntas_db = json.loads(response.text)

df = pd.DataFrame(ntas_db)

In [4]:
pkv = df.drop(['name','shape_area','population'],axis=1)


In [26]:
df = pd.DataFrame(columns=pkv["nta"].values,index=pkv["boro_name"].unique()).reset_index()


In [27]:
df

,index,QN08,BX28,QN55,BK50,BX41,QN26,BK68,QN38,QN52,...,QN99,BX13,BX99,BK45,BK82,BK93,BK73,MN28,BK40,BK99
0,Queens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Brooklyn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Manhattan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Staten Island,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df =df.fillna(0)

In [29]:
for i, row in pkv.iterrows():
    df.loc[df['index']==row['boro_name'],row['nta']]=1



In [30]:
df

,index,QN08,BX28,QN55,BK50,BX41,QN26,BK68,QN38,QN52,...,QN99,BX13,BX99,BK45,BK82,BK93,BK73,MN28,BK40,BK99
0,Queens,1,0,1,0,0,1,0,1,1,...,1,0,0,0,0,0,0,0,0,0
1,Bronx,0,1,0,0,1,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
2,Brooklyn,0,0,0,1,0,0,1,0,0,...,0,0,0,1,1,1,1,0,1,1
3,Manhattan,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Staten Island,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
df.to_csv("../aux_data/pkv.csv")